# Tips for writing HTTP server
## Web server에 있는 파일 위치와 읽기
Web server는 contents가 저장된 root directory를 지정할 수 있다. 편의상 source program과 같은 directory로 하자. Request line에서 `/test/index.html` 이라는 path가 있다면, web server는 `./test/index.html` 파일을 찾아 읽으면 된다. 

Response message의 첫 줄, status line은 
  - if the file exists, `HTTP/1.1 200 OK`
  - otherwise, `HTTP/1.1 404 Not found`

파일은 binary mode로 읽어야 한다(읽은 결과는 `bytes` type이다.

## Response message의 header들  정의  
`Date`, `Server`, `Connection` header는 반드시 만들어야 한다. Request message의 `Connection` header에 따라 Response의 `Connection` header를 `keep-alive`(default) 또는 `close`로 설정해야 한다.

File이 존재할 경우 `Accept-range`, `Content-type`, `Content-length` header를 삽입해야 한다.

In [6]:
import mimetypes
import time

res_headers = {}
res_headers['Date'] = time.asctime()
res_headers['Server'] = 'MyServer/1.0'
res_headers['Accept-range'] = 'bytes'
res_headers['Content-type'] = mimetypes.guess_type('/test/s3test2.gif')[0]
res_headers

{'Date': 'Mon Oct 14 20:37:34 2019',
 'Server': 'MyServer/1.0',
 'Accept-range': 'bytes',
 'Content-type': 'image/gif'}

## `socketserver` module 활용해서 server 구축하기

`socketserver.TCPServer` class를 이용하면, socket programming detail을 몰라도 구현할 수 있다. 이때 여러분은 `StreamRequestHandler` class를 상속받아 `handle` method만 implement하면 충분하다.

`TCPServer` class에서 `accept`한 connected socket을 `makefile`로 만들어 아래 `HTTPHandler` class로 pass해 준다. File-like stream object은 다음과 같이 access할 수 있다.
  - incoming stream: `self.rfile`
  - outgoing stream: `self.wfile`
    - 예) `self.wfile.write(message)

AS-1에서 작성한 headers.py 파일을 import해서 header를 처리하라.

In [ ]:
import socketserver
import time         # to know current time
import mimetypes    # to map filenames to MIME types
import headers      # your module 'headers.py'

class HTTPHandler(socketserver.StreamRequestHandler):
    def handle(self):
        # read a request message
        pass
        # read content from file name: '.' + path
        pass
        # Build the response message
        pass
        self.wfile.flush()  # Flush-out output buffer for immediate sending
    
# create a TCPServer object with your handler class
http_server = socketserver.TCPServer(('', 8080), HTTPHandler)
http_server.serve_forever()

## Testing your HTTP server
먼저 HTTP server를 실행한 후에, client로서 browser나 `urllib.request.urlopen` 함수를 사용할 수 있다.
### Browser에서
Chrome browser에서 '개발자 도구'를 띄우고 다음과 같이 url을 입력하자.
```http://localhost:8080/test/index.html
```

 Browser는 base html file을 `GET`한 후, 이를 parsing하고, 연속해서 image object들을 fetch하기 위해 `GET` request message를 자동적으로  생성하여 송신한다. 여러분은 server에서 읽은 request message를 print해서 확인할 수 있을 것이다. 이때, `Connection` header는 keep-alive (Persistent connection mode)임을 확인 할 수 있다.
 
### urllib module 활용
```Python
from urllib.request import urlopen

f = urlopen(url)
content = f.read()
```

위 방법은 request header는 `Connection: close`로 설정하여 non-persistent mode로 연결 요청하는 방식이다. 이후 각각의 image들에 대해 같은 방법으로 fetch해 오는 code를 작성해야 한다.